In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
news = pd.read_csv('lenta_2020-01-01_2024-10-10.csv')[[ "title","text",'rightcol', 'snippet']]

In [3]:
from tqdm import tqdm
tqdm.pandas()

In [4]:
import re
import nltk
from nltk.corpus import stopwords
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)


# nltk.download('stopwords')
russian_stopwords = set(stopwords.words("russian"))
def preprocess_russian_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    lemmatized_words = [_.lemma for _ in doc.tokens]
    cleaned_text = [word for word in lemmatized_words if word not in russian_stopwords]

    return ' '.join(cleaned_text)



In [5]:
for i in news:
    news[i]=news[i].apply(str).progress_apply(preprocess_russian_text)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68330/68330 [03:39<00:00, 311.26it/s]


In [6]:
news.to_csv('prepare.csv')
news = pd.read_csv('prepare.csv')[[ "title","text",'rightcol', 'snippet']]
news = news.loc[np.random.permutation(len(news))[:10000]]

In [7]:
news

,title,text,rightcol,snippet
44874,россиянин предупредить подорожание бытовой тех...,фото олег харсей коммерсантъ влад горячев опро...,россиянин предупредить подорожание бытовой тех...,фото олег харсей коммерсантъ влад цена несколь...
18510,александр ведяхин максима орешкин открыть конф...,кадр ai journey григорий шугай первый зампред ...,александр ведяхин максима орешкин открыть конф...,кадр ai journey григорий шугай первый интеллек...
64458,роснефть ставропольский край договориться разв...,фото simona pilolla 2 shutterstock fotodom гри...,роснефть ставропольский край договориться разв...,фото simona pilolla 2 shutterstock развитие ав...
46618,россиянин оценить возможный расход учеба вуз,фото игорь иванко коммерсантъ дмитрия воронин ...,россиянин оценить возможный расход учеба вуз,фото игорь иванко коммерсантъ дмитрия провести...
42165,газпром сообщить завершение проектирование нов...,фото александра чиженок коммерсантъ платон щук...,газпром сообщить завершение проектирование нов...,фото александра чиженок коммерсантъ выполнятьс...
...,...,...,...,...
44923,казахстан узбекистан приостановить работа росс...,фото светлана шевченко риа новость влад горяче...,казахстан узбекистан приостановить работа росс...,фото светлана шевченко риа новость платежный с...
64777,трата россиянин популярный лакомство вырасти,фото михаил воскресенский риа новость кирилл л...,трата россиянин популярный лакомство вырасти,фото михаил воскресенский риа новость аналогич...
6543,польский компания договориться условие покупка...,фото пресс служба администрация президент укра...,pgnig фги подписать договор давать инвестор ши...,фото пресс служба администрация киев договор к...
4805,составить рейтинг самый дорогой бренд мир,фото feng li getty images американский корпора...,facebook опередить компания coca cola,фото feng li getty images список самый дорогой...


In [8]:
from gensim import corpora
from gensim.models import LdaModel,LsiModel
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

def get_corpus(df):
    corpuses = []
    dictionares =[]
    for i in df:
        texts = [[word for word in simple_preprocess(doc)] for doc in df[i].apply(str).tolist()]
        dictionary = corpora.Dictionary(texts)
        dictionares.append(dictionary)
        corpuses.append([dictionary.doc2bow(text) for text in texts])
    return corpuses,dictionares
def get_tfidf_embs(df):
    embs = []
    vecs = []
    for i in df:
        vec = TfidfVectorizer(max_features=1000)
        embs.append(vec.fit_transform(df[i].apply(str)))
        vecs.append(vec)
    return embs,vecs

def get_tems_lda(corpuses,dictionares, num_topics = 5):

    for corpus,dictionary in zip(corpuses,dictionares):
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
        print()

        for idx, topic in lda_model.print_topics(-1):
            print(f"Тема {idx + 1}: {topic}")
def get_tems_lsi(corpuses,dictionares, num_topics = 5):

    for corpus,dictionary in zip(corpuses,dictionares):
        lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=num_topics)
               
        print()

        for idx, topic in lsi_model.print_topics(-1):
            print(f"Тема {idx + 1}: {topic}")




In [9]:
def get_tems_lda_scikit(embs,vectorizers, num_topics = 5):

    for emb,vectorizer in zip(embs,vectorizers):
    
        lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
        lda.fit(emb)
        print()

        for topic_idx, topic_words in enumerate(lda.components_):
            top_words_idx = topic_words.argsort()[-10:][::-1]
            top_words = [vectorizer.get_feature_names_out()[i] for i in top_words_idx]
            print(f"Тема {topic_idx + 1}: {', '.join(top_words)}")
def get_tems_nmf_scikit(embs,vectorizers, num_topics = 5):

    for emb,vectorizer in zip(embs,vectorizers):
        nmf = NMF(n_components=num_topics, random_state=42)
        nmf.fit(emb)
        
        feature_names = vectorizer.get_feature_names_out()
        print()
        for topic_idx, topic_words in enumerate(nmf.components_):
            top_words_idx = topic_words.argsort()[-10:][::-1]
            top_words = [feature_names[i] for i in top_words_idx]
            print(f"Тема {topic_idx + 1}: {', '.join(top_words)}")

def get_tems_kmean(embs,documents, num_topics = 5):
# пояснения 
    for emb,col in zip(embs,documents):
        kmeans = KMeans(n_clusters=num_topics, random_state=42)
        kmeans.fit(emb)
        print()
        for cluster_idx in range(num_topics):
            cluster_documents = [documents[col][i] for i, label in enumerate(kmeans.labels_) if label == cluster_idx]
            print(f"Кластер {cluster_idx + 1}:")
            for doc in cluster_documents:
                print(f" - {doc[:10]}")



In [10]:
corposes,dictionares = get_corpus(news)

In [11]:
embs,vecs = get_tfidf_embs(news)

In [12]:
get_tems_lda(corposes,dictionares)


Тема 1: 0.047*"россия" + 0.032*"газ" + 0.020*"европа" + 0.019*"поставка" + 0.015*"цена" + 0.015*"нефть" + 0.015*"российский" + 0.012*"газпром" + 0.011*"китай" + 0.011*"украина"
Тема 2: 0.079*"назвать" + 0.027*"россия" + 0.022*"российский" + 0.017*"самый" + 0.013*"северный" + 0.013*"поток" + 0.012*"сбер" + 0.010*"срок" + 0.009*"мир" + 0.008*"новый"
Тема 3: 0.067*"россиянин" + 0.020*"рассказать" + 0.013*"предупредить" + 0.013*"банк" + 0.012*"раскрыть" + 0.010*"стать" + 0.010*"пенсия" + 0.009*"сбер" + 0.009*"кредит" + 0.008*"доход"
Тема 4: 0.064*"россия" + 0.017*"российский" + 0.016*"страна" + 0.016*"стать" + 0.016*"санкция" + 0.016*"сша" + 0.012*"новый" + 0.010*"россиянин" + 0.010*"против" + 0.009*"китайский"
Тема 5: 0.031*"россия" + 0.030*"цена" + 0.026*"рост" + 0.025*"рубль" + 0.024*"российский" + 0.021*"экономика" + 0.020*"доллар" + 0.017*"оценить" + 0.013*"курс" + 0.010*"нефть"

Тема 1: 0.015*"россия" + 0.013*"страна" + 0.013*"российский" + 0.012*"год" + 0.011*"это" + 0.008*"газ" + 

In [13]:
get_tems_lda(corposes,dictionares,11)


Тема 1: 0.087*"россия" + 0.028*"вырасти" + 0.025*"резко" + 0.023*"российский" + 0.016*"актив" + 0.016*"китайский" + 0.016*"продажа" + 0.013*"доход" + 0.012*"банк" + 0.011*"компания"
Тема 2: 0.042*"россия" + 0.038*"банк" + 0.030*"сбер" + 0.026*"новый" + 0.019*"российский" + 0.017*"против" + 0.015*"санкция" + 0.013*"бизнес" + 0.013*"запустить" + 0.012*"рассказать"
Тема 3: 0.043*"северный" + 0.041*"поток" + 0.025*"глава" + 0.021*"кремль" + 0.020*"санкция" + 0.017*"германия" + 0.013*"сделка" + 0.013*"сбер" + 0.012*"ответить" + 0.012*"рассказать"
Тема 4: 0.092*"цена" + 0.056*"рост" + 0.045*"россия" + 0.025*"россиянин" + 0.024*"предупредить" + 0.023*"нефть" + 0.023*"год" + 0.016*"спрогнозировать" + 0.015*"рекордный" + 0.015*"предречь"
Тема 5: 0.077*"рубль" + 0.060*"доллар" + 0.040*"курс" + 0.029*"миллиард" + 0.018*"евро" + 0.017*"миллион" + 0.010*"тысяча" + 0.009*"цифровой" + 0.008*"снизиться" + 0.008*"американский"
Тема 6: 0.030*"роснефть" + 0.023*"решение" + 0.020*"пообещать" + 0.019*"буд

In [14]:
get_tems_lsi(corposes,dictionares)


Тема 1: -0.925*"россия" + -0.155*"назвать" + -0.119*"цена" + -0.095*"нефть" + -0.082*"российский" + -0.081*"газ" + -0.076*"оценить" + -0.074*"рост" + -0.066*"новый" + -0.062*"сша"
Тема 2: 0.725*"российский" + 0.345*"назвать" + 0.293*"россиянин" + -0.277*"россия" + 0.180*"цена" + 0.166*"газ" + 0.144*"нефть" + 0.109*"европа" + 0.102*"рост" + 0.073*"экономика"
Тема 3: -0.711*"россиянин" + 0.486*"российский" + -0.419*"назвать" + 0.098*"газ" + 0.095*"нефть" + -0.086*"предупредить" + -0.075*"самый" + -0.052*"способ" + 0.052*"европа" + 0.051*"россия"
Тема 4: -0.774*"назвать" + 0.506*"россиянин" + 0.177*"цена" + -0.122*"самый" + 0.121*"рост" + 0.111*"предупредить" + 0.088*"стать" + 0.076*"рассказать" + -0.068*"срок" + 0.065*"нефть"
Тема 5: -0.705*"цена" + 0.343*"российский" + -0.275*"рост" + -0.267*"нефть" + -0.252*"газ" + 0.210*"россиянин" + -0.191*"европа" + 0.139*"россия" + 0.093*"банк" + -0.071*"назвать"

Тема 1: 0.436*"год" + 0.288*"процент" + 0.263*"это" + 0.216*"который" + 0.209*"компа

In [15]:
get_tems_lsi(corposes,dictionares,11)


Тема 1: -0.925*"россия" + -0.155*"назвать" + -0.119*"цена" + -0.095*"нефть" + -0.082*"российский" + -0.081*"газ" + -0.076*"оценить" + -0.074*"рост" + -0.066*"новый" + -0.062*"сша"
Тема 2: 0.725*"российский" + 0.345*"назвать" + 0.293*"россиянин" + -0.277*"россия" + 0.180*"цена" + 0.166*"газ" + 0.144*"нефть" + 0.109*"европа" + 0.102*"рост" + 0.073*"экономика"
Тема 3: 0.711*"россиянин" + -0.486*"российский" + 0.419*"назвать" + -0.098*"газ" + -0.095*"нефть" + 0.086*"предупредить" + 0.075*"самый" + 0.053*"способ" + -0.052*"европа" + -0.051*"россия"
Тема 4: -0.775*"назвать" + 0.506*"россиянин" + 0.177*"цена" + -0.122*"самый" + 0.121*"рост" + 0.111*"предупредить" + 0.088*"стать" + 0.076*"рассказать" + -0.068*"срок" + 0.065*"нефть"
Тема 5: -0.705*"цена" + 0.343*"российский" + -0.275*"рост" + -0.267*"нефть" + -0.252*"газ" + 0.210*"россиянин" + -0.191*"европа" + 0.139*"россия" + 0.093*"банк" + -0.071*"назвать"
Тема 6: -0.687*"газ" + -0.456*"европа" + 0.290*"цена" + 0.260*"нефть" + -0.170*"поста

In [16]:
get_tems_lda_scikit(embs,vecs, num_topics = 5)


Тема 1: цена, нефть, россия, газ, европа, северный, поток, российский, подорожать, германия
Тема 2: россиянин, назвать, россия, предупредить, способ, самый, пенсия, новый, российский, рассказать
Тема 3: россия, российский, газ, поставка, страна, украина, китай, нефть, госдума, запретить
Тема 4: рубль, банк, доллар, россиянин, стать, сбер, россия, начать, миллиард, курс
Тема 5: экономика, россия, рост, предсказать, назвать, оценить, российский, сша, рынок, мировой

Тема 1: россия, российский, страна, санкция, компания, это, газ, который, сша, украина
Тема 2: доллар, нефть, процент, год, цена, газ, миллион, миллиард, поставка, баррель
Тема 3: компания, который, год, сбер, весь, новый, это, сервис, проект, сеть
Тема 4: процент, год, рост, цена, экономика, инфляция, рубль, ставка, валюта, доллар
Тема 5: процент, рубль, россиянин, год, тысяча, пенсия, выплата, зарплата, это, квартира

Тема 1: россия, российский, газ, европа, поставка, крупный, китай, производитель, город, оценить
Тема 2: р

In [17]:
get_tems_lda_scikit(embs,vecs, num_topics = 11)


Тема 1: северный, поток, газ, россия, российский, поставка, нефть, призвать, цена, германия
Тема 2: россиянин, назвать, предупредить, самый, россия, зарплата, способ, пенсия, автомобиль, китайский
Тема 3: россия, санкция, найти, против, ввести, российский, предложить, новый, суд, страна
Тема 4: рубль, доллар, курс, миллиард, евро, миллион, россиянин, кредит, назвать, условие
Тема 5: экономика, предсказать, мировой, рынок, рост, сша, пообещать, кризис, российский, предречь
Тема 6: сбер, рассказать, российский, россия, банк, роснефть, новый, появиться, деньга, регион
Тема 7: газ, нефть, цена, европа, рекорд, сша, китай, причина, россия, газпром
Тема 8: россия, начать, российский, мир, богатый, человек, страна, производство, запретить, москвич
Тема 9: россия, ставка, число, взлететь, цена, год, россиянин, дать, ключевой, снизить
Тема 10: россия, упасть, рухнуть, доход, резко, продажа, нефть, американский, торговля, акция
Тема 11: россия, подорожать, российский, срок, город, назвать, газ,

In [18]:
get_tems_nmf_scikit(embs,vecs, num_topics = 5)


Тема 1: россия, оценить, новый, стать, поставка, начать, сша, предложить, продажа, резко
Тема 2: россиянин, предупредить, стать, рассказать, пенсия, деньга, подорожание, зарплата, кредит, новый
Тема 3: российский, компания, банк, экономика, сша, город, санкция, рынок, газ, регион
Тема 4: назвать, самый, способ, срок, хороший, экономика, причина, условие, рубль, главный
Тема 5: цена, рост, газ, нефть, европа, доллар, предсказать, курс, упасть, объяснить

Тема 1: компания, который, россия, это, банк, год, новый, российский, весь, также
Тема 2: газ, поставка, европа, газпром, кубометр, поток, газопровод, северный, европейский, украина
Тема 3: процент, год, рост, цена, вырасти, тысяча, инфляция, россиянин, показатель, уровень
Тема 4: нефть, баррель, цена, доллар, добыча, сша, российский, страна, опек, миллион
Тема 5: рубль, доллар, валюта, курс, миллиард, евро, торг, биржа, банк, валютный

Тема 1: россия, оценить, стать, новый, поставка, продажа, начать, сша, предложить, резко
Тема 2: рос

In [19]:
get_tems_nmf_scikit(embs,vecs, num_topics = 11)


Тема 1: россия, резко, вырасти, предложить, продажа, подорожать, поставка, автомобиль, заявить, оценить
Тема 2: россиянин, предупредить, пенсия, подорожание, деньга, зарплата, кредит, рост, рассказать, доход
Тема 3: российский, компания, город, регион, нефть, рынок, банк, миллиардер, бизнес, власть
Тема 4: назвать, самый, срок, способ, хороший, причина, условие, главный, популярный, мир
Тема 5: цена, нефть, рост, потолок, бензин, вырасти, взлететь, снижение, продукт, рекорд
Тема 6: рубль, курс, доллар, миллиард, евро, упасть, экономист, превысить, низкий, спрогнозировать
Тема 7: газ, европа, поставка, газпром, украина, отказ, польша, германия, заявить, транзит
Тема 8: экономика, оценить, путин, мировой, рост, влияние, предсказать, заявить, европа, главный
Тема 9: сша, санкция, против, северный, поток, китай, ввести, нефть, угроза, заявить
Тема 10: стать, известный, меньше, мир, частый, деньга, покупать, редкий, банк, план
Тема 11: новый, рассказать, сбер, начать, банк, запустить, рабо